In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import json
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # For better error messages

In [ ]:
# Configuration
MODEL_NAME = "ai4bharat/IndicBART"
#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH = 128
BATCH_SIZE = 8

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Add style tokens
STYLE_TOKENS = ["<formal>", "<informal>"]
tokenizer.add_tokens(STYLE_TOKENS)
original_vocab_size = model.config.vocab_size
model.resize_token_embeddings(len(tokenizer))


# Step 4: Update config and move to GPU
model.config.vocab_size = len(tokenizer)
device = torch.device("cuda")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/976M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:

class HindiHumanizationDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=128):
        with open(data_path) as f:
            self.data = [json.loads(line) for line in f]

        self.formal_informal_pairs = []
        for item in self.data:
            for ann in item['annotations']:
                if ann['formal_sent'] == 'candidate_1':
                    self.formal_informal_pairs.append({
                        'formal': item['candidate_1'],
                        'informal': item['candidate_0']
                    })

        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.formal_informal_pairs)

    def __getitem__(self, idx):
        pair = self.formal_informal_pairs[idx]
        source = f"<formal>{pair['formal']}</formal>"
        target = f"<informal>{pair['informal']}</informal>"

        encoding = self.tokenizer(
            source,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        target_encoding = self.tokenizer(
            target,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
device = torch.device("cuda")

# Dataset preparation
dataset = HindiHumanizationDataset('/content/hi_annotations.jsonl', tokenizer, MAX_LENGTH)
train_data, val_data = train_test_split(dataset, test_size=0.1)

# Training setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)
# Start training
trainer.train()



In [ ]:
device = torch.device("cuda")

import re

def humanize_hindi(text, style_strength=0.8):
    inputs = tokenizer(
        f"<formal>{text}",
        return_tensors="pt",
        max_length=MAX_LENGTH,
        truncation=True
    )

    # Block English characters and special symbols
    bad_words = ["/", "&", "the", "and", "or", "error-mask", "[CLS]"]
    bad_word_ids = [tokenizer.encode(word, add_special_tokens=False) for word in bad_words]


    outputs = model.generate(
          inputs.input_ids,
          attention_mask=inputs.attention_mask,
          max_length=min(len(inputs.input_ids[0]) + 15, 100),  # Tighter length control
          do_sample=True,
          temperature=0.75,
          top_k=30,
          top_p=0.9,
          repetition_penalty=2.5,  # Stronger anti-repeat
          no_repeat_ngram_size=4,   # Prevent 4-word repeats
          num_beams=3,
          length_penalty=0.4,       # Force concise outputs
          early_stopping=True,
          eos_token_id=tokenizer.eos_token_id,
          pad_token_id=tokenizer.pad_token_id,
          bad_words_ids=bad_word_ids  # Block unwanted tokens
      )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return enhanced_hindi_cleanup(decoded)

def enhanced_hindi_cleanup(text):
    # Remove all non-Devanagari characters except basic punctuation
    text = re.sub(r'[^\u0900-\u097F\u097F\uA8E0-\uA8FF\u1CD0-\u1CFF\s।,!?-]', '', text)

    # Hindi-specific normalizations
    replacements = {
        r'\s+': ' ',
        r'(।)\1+': '।',  # Multiple dandas
        r'([\u0900-\u097F]+)([\s,|]+)(\1([\s,|]+))+': r'\1\2',  # Word repetitions
        r'([क-ह]़)़': r'\1',  # Double nuktas
        r'ॉ': 'ो',  # Matra correction
        r'(\S)़': r'\1़'  # Nukta positioning
    }

    for pattern, repl in replacements.items():
        text = re.sub(pattern, repl, text)

    # Select best sentence
    sentences = [s.strip() for s in text.split('।') if 10 < len(s) < 60]
    return '। '.join(sentences[:1]) + '।' if sentences else text


In [ ]:
# Example usage
# formal_text = "आप कैसे हैं?"
#formal_text = "नमस्ते!"
#formal_text = "बिल्कुल! बोलिए, मैं मदद करने के लिए तैयार हूँ। "
formal_text = " मैं एक AI हूँ, जिसका उद्देश्य आपकी मदद करना है। मैं आपकी कोडिंग, मशीन लर्निंग, डेटा साइंस, लेखन और अन्य विषयों से जुड़ी समस्याओं को हल करने में सहायता कर सकता हूँ। इसके अलावा, मैं आपके प्रोजेक्ट्स के लिए सुझाव दे सकता हूँ, रिपोर्ट तैयार कर सकता हूँ, और आपकी पढ़ाई या शोध में भी मदद कर सकता हूँ। 😎"
#formal_text = "अब आप अपने बारे में कुछ बताएँ! 😄"
#formal_text = "भारतीय टेस्ट टीम में शामिल इस गेंदबाज ने आईपीएल-2022 में बंगलूरू के लिए पदार्पण किया था। आकाशदीप को तीन सीजन में आरसीबी की ओर से 8 मैच खेलने का मौका मिला, जिसमें सात विकेट लेने में कामयाब रहे।"
#formal_text = "महिला क्रिकेट टीम की कप्तान मिताली राज ने भी हरभजन को बधाई दी"
#formal_text = "औपचारिक पत्र उन्हें लिखा जाता है जिनसे हमारा कोई निजी संबंध ना हो। व्यवसाय से संबंधी, प्रधानाचार्य को लिखे प्रार्थना पत्र, आवेदन पत्र, सरकारी विभागों को लिखे गए पत्र, संपादक के नाम पत्र आदि औपचारिक-पत्र कहलाते हैं। औपचारिक पत्रों की भाषा सहज और शिष्टापूर्ण होती है। इन पत्रों में केवल काम या अपनी समस्या के बारे में ही बात कही जाती है।"
humanized = humanize_hindi(formal_text)
print(f"Formal: {formal_text}")
print(f"Humanized: {humanized}")

Formal:  मैं एक AI हूँ, जिसका उद्देश्य आपकी मदद करना है। मैं आपकी कोडिंग, मशीन लर्निंग, डेटा साइंस, लेखन और अन्य विषयों से जुड़ी समस्याओं को हल करने में सहायता कर सकता हूँ। इसके अलावा, मैं आपके प्रोजेक्ट्स के लिए सुझाव दे सकता हूँ, रिपोर्ट तैयार कर सकता हूँ, और आपकी पढ़ाई या शोध में भी मदद कर सकता हूँ। 😎
Humanized: आपको मैं एक हूँ, जिसका उद्देश्य आपकी मदद करना है।


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp -r logs /content/drive/MyDrive/2/
!cp -r results /content/drive/MyDrive/2/
